In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
 import os, shutil
import numpy as np

from matplotlib import image, pyplot
from skimage.transform import resize


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc 
from sklearn.svm import SVC
from sklearn import svm

**For western/ ethenic classification**

In [6]:
DATA_PATH = '/content/drive/MyDrive/updatedDataset/'
westEth=[]
labelwe=[]
hashList=[]
count=2
for folder in os.listdir(DATA_PATH):
    if folder=="Gender":
      continue;
    print(">>>Reading ",folder)
    count-=1
    print(count)
    
    for file in os.listdir(DATA_PATH+folder):
         if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
              westEth.append(resize(img, (156, 156, 3)))
              hashList.append(hsh)
              labelwe.append(count)
westEth=np.array(westEth)
labelwe=np.array(labelwe)

>>>Reading  western
1
>>>Reading  ethenic
0


In [8]:
resultPath = '/content/drive/MyDrive/Resultse'
train_folder = os.listdir(DATA_PATH).remove("Gender")

In [9]:
print("Number of Ethenic",len(labelwe[labelwe==0]))
print("Number of western",len(labelwe[labelwe==1]))

Number of Ethenic 1715
Number of western 1873


In [10]:
print("western/ethenic data shape : ",westEth.shape," Label shape : ",labelwe.shape)

western/ethenic data shape :  (3588, 156, 156, 3)  Label shape :  (3588,)


In [11]:
testPercentage=0.3
x_train,x_test,y_train,y_test = train_test_split(westEth,labelwe,test_size = testPercentage,random_state=50, stratify=labelwe,shuffle=True)

In [12]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (2511, 156, 156, 3)  y_train shape :  (2511,)
x_test shape :  (1077, 156, 156, 3)  y_test shape :  (1077,)


In [13]:
print("Number of train ethenic",len(y_train[y_train==0]))
print("Number of train western",len(y_train[y_train==1]))

print("Number of test  ethenic",len(y_test[y_test==0]))
print("Number of test western",len(y_test[y_test==1]))

Number of train ethenic 1200
Number of train western 1311
Number of test  ethenic 515
Number of test western 562


In [14]:
NO_OF_EPOCHS=100
BATCH_SIZE=32

model2=Sequential()
model2.add(Conv2D(32,kernel_size=3, padding='same',activation='relu',input_shape=(156, 156,3)))
model2.add(MaxPool2D(pool_size=(2, 2)))
model2.add(Conv2D(64,kernel_size=3, padding='same',activation='relu'))
model2.add(MaxPool2D(pool_size=(2, 2)))
model2.add(Conv2D(128,kernel_size=3, padding='same',activation='relu'))
model2.add(Dropout(0.1))
model2.add(Flatten())
model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(1,activation='sigmoid'))
model2.summary()
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 156, 156, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 78, 78, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 78, 78, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 39, 39, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 39, 128)       73856     
                                                                 
 dropout (Dropout)           (None, 39, 39, 128)       0

In [ ]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=2, min_lr=0.001)
model2.fit(x_train,y_train,epochs=NO_OF_EPOCHS,verbose=1,batch_size=BATCH_SIZE,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 0.9055 - accuracy: 0.5337
Epoch 1: val_accuracy improved from -inf to 0.52089, saving model to /content/drive/MyDrive/Resultse/checkpoint-0001.hdf5
79/79 [==============================] - 158s 2s/step - loss: 0.9055 - accuracy: 0.5337 - val_loss: 0.7024 - val_accuracy: 0.5209 - lr: 0.0010
Epoch 2/100
79/79 [==============================] - ETA: 0s - loss: 0.6856 - accuracy: 0.5508
Epoch 2: val_accuracy improved from 0.52089 to 0.52182, saving model to /content/drive/MyDrive/Resultse/checkpoint-0002.hdf5
79/79 [==============================] - 150s 2s/step - loss: 0.6856 - accuracy: 0.5508 - val_loss: 0.7683 - val_accuracy: 0.5218 - lr: 0.0010
Epoch 3/100
79/79 [==============================] - ETA: 0s - loss: 0.6671 - accuracy: 0.5890
Epoch 3: val_accuracy improved from 0.52182 to 0.58960, saving model to /content/drive/MyDrive/Resultse/checkpoint-0003.hdf5
79/79 [==============================] - 152s 2s/step - l

In [16]:
model2.load_weights(resultPath + "/checkpoint-0014.hdf5")

In [17]:
prediction_prob1 = model2.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

34/34 [==============================] - 16s 439ms/step


In [15]:
m1model = Model(inputs=model2.input,outputs=model2.get_layer('dense_1').output)

In [18]:
m1_x_train = m1model.predict(x_train,verbose=1)
m1_x_test = m1model.predict(x_test,verbose=1)


34/34 [==============================] - 14s 424ms/step


In [19]:
print("Shape of model1 Train and Test DF : ",m1_x_train.shape," : ",m1_x_test.shape)


Shape of model1 Train and Test DF :  (2511, 1)  :  (1077, 1)


In [20]:
def printMetrics(true,pred):
    print("Accuracy : ",accuracy_score(true, pred))
    print("Precision",precision_score(true, pred , average="weighted"))
    print("Recall : ",recall_score(true, pred , average="weighted"))
    print("F1-score : ",f1_score(true, pred, average="weighted"))
    print("Confusion Matrix : ")
    print(confusion_matrix(true, pred))
    print(classification_report(true,pred))

In [21]:
RSVM = svm.SVC(kernel='rbf',probability=True)
RSVM.fit(m1_x_train, y_train)
RSVMprob = RSVM.predict_proba(m1_x_test)
y_pred = RSVM.predict(m1_x_test)
printMetrics(y_test,y_pred)

Accuracy :  0.6555246053853296
Precision 0.6559783259632568
Recall :  0.6555246053853296
F1-score :  0.6556560013743798
Confusion Matrix : 
[[336 179]
 [192 370]]
              precision    recall  f1-score   support

           0       0.64      0.65      0.64       515
           1       0.67      0.66      0.67       562

    accuracy                           0.66      1077
   macro avg       0.66      0.66      0.66      1077
weighted avg       0.66      0.66      0.66      1077



**For gender classification**


In [10]:
DATA_PATH = '/content/drive/MyDrive/updatedDataset/Gender/'
genderData=[]
glabel=[]
ghashList=[]
count=2
for folder in os.listdir(DATA_PATH):
    print(">>>Reading ",folder)
    count-=1
    print(count)
    
    for file in os.listdir(DATA_PATH+folder):
         if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in ghashList):
              genderData.append(resize(img, (156, 156, 3)))
              ghashList.append(hsh)
              glabel.append(count)
genderData=np.array(genderData)
glabel=np.array(glabel)

>>>Reading  female
1
>>>Reading  male
0


In [20]:
resultPath = '/content/drive/MyDrive/gResult'
train_folder = os.listdir(DATA_PATH)

In [21]:
print("Number of maleGarments",len(glabel[glabel==0]))
print("Number of femaleGarments",len(glabel[glabel==1]))

Number of maleGarments 1779
Number of femaleGarments 1843


In [22]:
print("Gender data shape : ",genderData.shape," Label shape : ",glabel.shape)

Gender data shape :  (3622, 156, 156, 3)  Label shape :  (3622,)


In [23]:
testPercentage=0.3
x_train,x_test,y_train,y_test = train_test_split(genderData,glabel,test_size = testPercentage,random_state=50, stratify=glabel,shuffle=True)

In [24]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (2535, 156, 156, 3)  y_train shape :  (2535,)
x_test shape :  (1087, 156, 156, 3)  y_test shape :  (1087,)


In [25]:
print("Number of train male male",len(y_train[y_train==0]))
print("Number of train female garments",len(y_train[y_train==1]))

print("Number of test male garments ",len(y_test[y_test==0]))
print("Number of test female garments",len(y_test[y_test==1]))

Number of train male male 1245
Number of train female garments 1290
Number of test male garments  534
Number of test female garments 553


In [26]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=2, min_lr=0.001)
model2.fit(x_train,y_train,epochs=NO_OF_EPOCHS,verbose=1,batch_size=BATCH_SIZE,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/100
80/80 [==============================] - ETA: 0s - loss: 0.9923 - accuracy: 0.5093
Epoch 1: val_accuracy improved from -inf to 0.60626, saving model to /content/drive/MyDrive/gResult/checkpoint-0001.hdf5
80/80 [==============================] - 113s 1s/step - loss: 0.9923 - accuracy: 0.5093 - val_loss: 0.6850 - val_accuracy: 0.6063 - lr: 0.0010
Epoch 2/100
80/80 [==============================] - ETA: 0s - loss: 0.6579 - accuracy: 0.6114
Epoch 2: val_accuracy did not improve from 0.60626
80/80 [==============================] - 108s 1s/step - loss: 0.6579 - accuracy: 0.6114 - val_loss: 0.6576 - val_accuracy: 0.6035 - lr: 0.0010
Epoch 3/100
80/80 [==============================] - ETA: 0s - loss: 0.5527 - accuracy: 0.7101
Epoch 3: val_accuracy improved from 0.60626 to 0.65501, saving model to /content/drive/MyDrive/gResult/checkpoint-0003.hdf5
80/80 [==============================] - 111s 1s/step - loss: 0.5527 - accuracy: 0.7101 - val_loss: 0.6676 - val_accuracy: 0.6550 - l

In [27]:
model2.load_weights(resultPath + "/checkpoint-0043.hdf5")

In [28]:
prediction_prob1 = model2.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

34/34 [==============================] - 11s 333ms/step


In [30]:
m1model = Model(inputs=model2.input,outputs=model2.get_layer('dense_1').output)

In [31]:
m1_x_train = m1model.predict(x_train,verbose=1)
m1_x_test = m1model.predict(x_test,verbose=1)

34/34 [==============================] - 13s 364ms/step


In [32]:
print("Shape of model1 Train and Test DF : ",m1_x_train.shape," : ",m1_x_test.shape)

Shape of model1 Train and Test DF :  (2535, 1)  :  (1087, 1)


In [33]:
def printMetrics(true,pred):
    print("Accuracy : ",accuracy_score(true, pred))
    print("Precision",precision_score(true, pred , average="weighted"))
    print("Recall : ",recall_score(true, pred , average="weighted"))
    print("F1-score : ",f1_score(true, pred, average="weighted"))
    print("Confusion Matrix : ")
    print(confusion_matrix(true, pred))
    print(classification_report(true,pred))

In [34]:
RSVM = svm.SVC(kernel='rbf',probability=True)
RSVM.fit(m1_x_train, y_train)
RSVMprob = RSVM.predict_proba(m1_x_test)
y_pred = RSVM.predict(m1_x_test)
print("cost-insensitive")
printMetrics(y_test,y_pred)

cost-insensitive
Accuracy :  0.7175712971481141
Precision 0.7176709718188835
Recall :  0.7175712971481141
F1-score :  0.7175913779920435
Confusion Matrix : 
[[384 150]
 [157 396]]
              precision    recall  f1-score   support

           0       0.71      0.72      0.71       534
           1       0.73      0.72      0.72       553

    accuracy                           0.72      1087
   macro avg       0.72      0.72      0.72      1087
weighted avg       0.72      0.72      0.72      1087

